# Анализ данных

# <a id="0">Содержание</a>

- <a href="#1">Открытие данных и их описание</a>
- <a href="#2">Сборка датасета</a>  
    - <a href="#21">Агрегация по пользователям</a>
    - <a href="#22">Обработка категориальных и временных признаков</a>
    - <a href="#23">Поиск и обработка аномалий в данных</a>
    - <a href="#24">Добавление новых признаков</a>

In [2]:
# Импортируем необходимые библиотеки
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# <a id="1">Открытие данных и их описание</a>

Загрузим предобработанные данные

In [3]:
# Откроем данные
users = pd.read_csv('../data/users_preprocessed.csv')
history = pd.read_csv('../data/history_preprocessed.csv')
validate = pd.read_csv('../src/validate.tsv', sep='\t')
validate_answers = pd.read_csv('../src/validate_answers.tsv', sep='\t')

In [4]:
# Посмотрим информацию о датасете
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27137 entries, 0 to 27136
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   Unnamed: 0  27137 non-null  int64
 1   user_id     27137 non-null  int64
 2   sex         27137 non-null  int64
 3   age         27137 non-null  int64
 4   city_id     27137 non-null  int64
dtypes: int64(5)
memory usage: 1.0 MB


In [5]:
# Удалим лишний столбец
users = users.drop(columns='Unnamed: 0')

In [6]:
# Рассмотрим первые 5 строк
users.head()

,user_id,sex,age,city_id
0,0,2,19,0
1,1,1,0,1
2,2,2,24,2
3,3,1,20,3
4,4,2,29,4


`users`:
- `user_id` – уникальный идентификатор пользователя
- `sex` – указанный пользователем пол в анкете
- `age` – указанный пользователем в анкете возраст пользователя. 0 – не указан.
- `city_id` - указанный пользователем в анкете город проживания. 0 – не указан.

In [7]:
# Посмотрим информацию о датасете
history.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1069931 entries, 0 to 1069930
Data columns (total 5 columns):
 #   Column      Non-Null Count    Dtype  
---  ------      --------------    -----  
 0   Unnamed: 0  1069931 non-null  int64  
 1   hour        1069931 non-null  int64  
 2   cpm         1069931 non-null  float64
 3   publisher   1069931 non-null  int64  
 4   user_id     1069931 non-null  int64  
dtypes: float64(1), int64(4)
memory usage: 40.8 MB


In [8]:
# Удалим лишний столбец
history = history.drop(columns='Unnamed: 0')

In [9]:
# Рассмотрим первые 5 строк
history.head()

,hour,cpm,publisher,user_id
0,10,30.00,1,15661
1,8,41.26,1,8444
2,7,360.00,1,15821
3,18,370.00,1,21530
4,8,195.00,2,22148


`history`:
- `hour` – в какой час пользователь видел объявление
- `cpm` - цена показанного рекламного объявления в рекламном аукционе. Это значит, что на данном аукционе это была максимальная ставка. 
- `publisher` - площадка, на который пользователь видел рекламу
- `user_id` - уникальный идентификатор пользователя

In [10]:
# Посмотрим информацию о датасете
validate.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1008 entries, 0 to 1007
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   cpm            1008 non-null   float64
 1   hour_start     1008 non-null   int64  
 2   hour_end       1008 non-null   int64  
 3   publishers     1008 non-null   object 
 4   audience_size  1008 non-null   int64  
 5   user_ids       1008 non-null   object 
dtypes: float64(1), int64(3), object(2)
memory usage: 47.4+ KB


In [11]:
# Рассмотрим первые 5 строк
validate.head()

,cpm,hour_start,hour_end,publishers,audience_size,user_ids
0,220.0,1058,1153,"7,17",1906,"12,44,46,50,58,71,93,122,134,143,176,184,187,1..."
1,312.0,1295,1301,"3,18",1380,"29,81,98,102,165,167,195,205,218,231,242,263,3..."
2,70.0,1229,1249,"1,2,3,9,15,21",888,"12,23,25,29,45,85,92,124,156,190,272,334,456,5..."
3,240.0,1295,1377,"1,14",440,"44,122,187,209,242,255,312,345,382,465,513,524..."
4,262.0,752,990,"1,3,7,8",1476,"15,24,30,43,50,53,96,105,159,168,181,190,196,2..."


`validate`:
- `cpm` - для какой цены объявления нужно сделать прогноз
- `hour_start` - предположительное время запуска рекламного объявления
- `hour_end` - предположительное время остановки рекламного объявления. По итогу прогноз делается для рекламного объявление, которое будет запущено в период времени `[hour_start, hour_end]`
- `publishers` - на каких площадках объявление может быть показано
- `audience_size` - размер аудитории объявления, количество идентификаторов в поле `user_ids`
- `user_ids` – аудитория объявления – список пользователей, кому рекламодатель хочет показать объявление.

В датасете 1008 строк. Проупсков нет

# <a id="2">Сборка датасета</a>

## <a href="#21">Агрегация по пользователям</a>

Сгруппируем датасет с агрегацией по пользователям, где посчитаем общее количество просмотренной рекламы, срденюю, медианную, максимальную стоимость рекламы

In [12]:
# Агрегация по пользователям
# Общее количество объявлений, просмотренных пользователем
user_agg = history.groupby('user_id').agg(
    total_ads=('cpm', 'count'),
    avg_cpm=('cpm', 'mean'),
    median_cpm=('cpm', 'median'),
    max_cpm=('cpm', 'max')
).reset_index()

# Посмотрим что получилось
user_agg

,user_id,total_ads,avg_cpm,median_cpm,max_cpm
0,1,79,111.247848,90.000,361.52
1,3,7,321.588571,263.760,531.22
2,4,126,45.086429,30.245,255.36
3,5,9,91.055556,42.500,390.00
4,6,35,226.118857,234.000,496.95
...,...,...,...,...,...
24855,27764,41,179.386098,170.000,400.92
24856,27765,75,130.001333,110.640,455.00
24857,27766,52,138.974231,120.000,342.00
24858,27767,1,143.880000,143.880,143.88


Сделаем датасет с подсчетом рапределения просмотров рекламы на 24 часа для каждого пользователя

In [13]:
# Преобразуем нарастающие часы в часы суток
history['hour_of_day'] = history['hour'] % 24

# Проверим результат
history[['hour', 'hour_of_day']]

,hour,hour_of_day
0,10,10
1,8,8
2,7,7
3,18,18
4,8,8
...,...,...
1069926,382,22
1069927,360,0
1069928,381,21
1069929,383,23


In [14]:
# Расчет распределения часов просмотра рекламы
hour_distribution = history.groupby('user_id')['hour_of_day'].apply(
    lambda x: np.histogram(x, bins=24, range=(0, 23))[0]
).reset_index(name='hours_distribution')

# Преобразование гистограммы в отдельные колонки для часов
hour_columns = [f'hour_{i}' for i in range(24)]
hour_distribution_df = pd.DataFrame(hour_distribution['hours_distribution'].tolist(), columns=hour_columns)
hour_distribution = pd.concat([hour_distribution['user_id'], hour_distribution_df], axis=1)

# Рассмотрим полученный датасет
hour_distribution

,user_id,hour_0,hour_1,hour_2,hour_3,hour_4,hour_5,hour_6,hour_7,hour_8,...,hour_14,hour_15,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23
0,1,0,0,0,0,0,0,0,0,7,...,2,9,12,5,4,9,2,0,0,1
1,3,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,2,1,1
2,4,6,0,0,0,0,0,2,4,11,...,4,4,4,12,6,5,11,4,3,0
3,5,0,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0
4,6,0,0,0,0,0,0,2,4,3,...,7,3,2,1,1,0,2,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24855,27764,2,0,0,0,0,0,1,2,1,...,0,0,1,0,0,2,2,6,7,15
24856,27765,0,1,1,6,4,11,6,4,1,...,5,0,0,2,6,3,3,0,0,1
24857,27766,4,0,0,0,1,0,3,2,4,...,0,2,2,5,10,3,2,2,3,1
24858,27767,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Теперь объединим обе таблицы

In [15]:
# Объединение user_agg и hour_distribution
user_agg = user_agg.merge(hour_distribution, on='user_id', how='left')
user_agg

,user_id,total_ads,avg_cpm,median_cpm,max_cpm,hour_0,hour_1,hour_2,hour_3,hour_4,...,hour_14,hour_15,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23
0,1,79,111.247848,90.000,361.52,0,0,0,0,0,...,2,9,12,5,4,9,2,0,0,1
1,3,7,321.588571,263.760,531.22,0,0,0,0,0,...,1,0,0,0,0,0,0,2,1,1
2,4,126,45.086429,30.245,255.36,6,0,0,0,0,...,4,4,4,12,6,5,11,4,3,0
3,5,9,91.055556,42.500,390.00,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0
4,6,35,226.118857,234.000,496.95,0,0,0,0,0,...,7,3,2,1,1,0,2,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24855,27764,41,179.386098,170.000,400.92,2,0,0,0,0,...,0,0,1,0,0,2,2,6,7,15
24856,27765,75,130.001333,110.640,455.00,0,1,1,6,4,...,5,0,0,2,6,3,3,0,0,1
24857,27766,52,138.974231,120.000,342.00,4,0,0,0,1,...,0,2,2,5,10,3,2,2,3,1
24858,27767,1,143.880000,143.880,143.88,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## <a href="#22">Агрегация по площадкам (publisher)</a>

Теперь сделаем агрегацию по площадкам, где размещается объявление

In [16]:
# # Агрегация по площадкам
publisher_agg = history.groupby('publisher').agg(
    total_ads=('cpm', 'count'),
    avg_cpm=('cpm', 'mean'),
    median_cpm=('cpm', 'median'),
    max_cpm=('cpm', 'max')
).reset_index()

publisher_agg

,publisher,total_ads,avg_cpm,median_cpm,max_cpm
0,1,648727,139.258345,104.750,570.00
1,2,252678,154.469666,120.000,570.00
2,3,66699,136.993496,96.480,570.00
3,4,2272,89.444190,62.940,565.11
4,5,6762,144.780482,105.000,570.00
5,6,5762,82.412383,45.000,570.00
6,7,59246,157.320954,117.500,570.00
7,8,3599,111.002167,81.710,570.00
8,9,10092,94.828052,66.555,570.00
9,10,4105,94.286565,60.000,570.00


## <a href="#23">Сессии пользователей</a>

Выведем новый столбец в номером сессии `session_id`.

Из условия задачи:
- Система запоминает все объявления, который видел пользователь в течении сессии и не показывает их, даже если это самые дорогие объявления в аукционе.  Новая сессия начинается после отсутсвия показов рекламы пользователю в течении 6 часов.

In [17]:
# Сортировка данных для вычисления сессий
history = history.sort_values(by=['user_id', 'hour'])

# Вычисление сессий
history['time_diff'] = history.groupby('user_id')['hour'].diff().fillna(0)
history['new_session'] = (history['time_diff'] > 6).astype(int)
history['session_id'] = history.groupby('user_id')['new_session'].cumsum()
history

,hour,cpm,publisher,user_id,hour_of_day,time_diff,new_session,session_id
296,8,165.0,1,1,8,0.0,0,0
972,9,53.2,1,1,9,1.0,0,0
3297,9,36.0,1,1,9,0.0,0,0
6377,9,51.0,1,1,9,0.0,0,0
1049164,19,31.5,1,1,19,10.0,1,1
...,...,...,...,...,...,...,...,...
733215,1188,285.0,2,27768,12,46.0,1,13
740224,1188,205.0,12,27768,12,0.0,0,13
988910,1216,210.0,2,27768,16,28.0,1,14
875868,1410,179.8,2,27768,18,194.0,1,15


Далее рассмотрим сессии пользователей и тоже сделаем по ним агрециии !!!

In [18]:
# Агрегация сессий
session_stats = history.groupby(['user_id', 'session_id']).agg(
    session_ads=('cpm', 'count')
).reset_index()

# Агрегация сессий по пользователям
session_agg = session_stats.groupby('user_id').agg(
    avg_ads_per_session=('session_ads', 'mean'),
    max_ads_per_session=('session_ads', 'max')
).reset_index()

session_agg

,user_id,avg_ads_per_session,max_ads_per_session
0,1,2.135135,7
1,3,1.000000,1
2,4,2.000000,7
3,5,1.285714,2
4,6,1.521739,3
...,...,...,...
24855,27764,1.464286,3
24856,27765,1.785714,7
24857,27766,1.677419,6
24858,27767,1.000000,1


## <a href="#24">Добавление данных пользователей</a> 

In [19]:
# Посмотрим количество пользователей по полу
users['sex'].value_counts()

sex
1    14249
2    12860
0       28
Name: count, dtype: int64

Преобразуем в бинарные данные столбец по полу, так как всего 30 пользователей не указали пол. Также разделим пользователей по группам возрастов

In [20]:
# Добавление данных пользователей
users['sex_binary'] = (users['sex'] == 1).astype(int)
users['age_group'] = pd.cut(users['age'], bins=[0, 18, 35, 50, np.inf], labels=['0-18', '19-35', '36-50', '51+'], right=False)
users

,user_id,sex,age,city_id,sex_binary,age_group
0,0,2,19,0,0,19-35
1,1,1,0,1,1,0-18
2,2,2,24,2,0,19-35
3,3,1,20,3,1,19-35
4,4,2,29,4,0,19-35
...,...,...,...,...,...,...
27132,27764,1,38,295,1,36-50
27133,27765,2,30,79,0,19-35
27134,27766,2,21,1953,0,19-35
27135,27767,2,17,0,0,0-18


Также определим топ-10 городов по количеству пользователей, а остальные определим как `other`

In [21]:
# Группировка городов по частоте
city_counts = users['city_id'].value_counts()
top_cities = city_counts.nlargest(10).index
users['city_group'] = users['city_id'].apply(lambda x: x if x in top_cities else 'other')
users

,user_id,sex,age,city_id,sex_binary,age_group,city_group
0,0,2,19,0,0,19-35,0
1,1,1,0,1,1,0-18,other
2,2,2,24,2,0,19-35,other
3,3,1,20,3,1,19-35,3
4,4,2,29,4,0,19-35,other
...,...,...,...,...,...,...,...
27132,27764,1,38,295,1,36-50,other
27133,27765,2,30,79,0,19-35,other
27134,27766,2,21,1953,0,19-35,other
27135,27767,2,17,0,0,0-18,0


 ## <a href="#25">Объединение фичей</a> 

Объединим фичи с пользователями и с сессиями и сохраним. Отдельно сохраним датасет с площадками

In [22]:
# Объединение всех фичей
user_features = user_agg.merge(session_agg, on='user_id', how='left')
user_features = user_features.merge(users, on='user_id', how='left')

In [23]:
# Посмотрим полученный датасет
user_features

,user_id,total_ads,avg_cpm,median_cpm,max_cpm,hour_0,hour_1,hour_2,hour_3,hour_4,...,hour_22,hour_23,avg_ads_per_session,max_ads_per_session,sex,age,city_id,sex_binary,age_group,city_group
0,1,79,111.247848,90.000,361.52,0,0,0,0,0,...,0,1,2.135135,7,1.0,0.0,1.0,1.0,0-18,other
1,3,7,321.588571,263.760,531.22,0,0,0,0,0,...,1,1,1.000000,1,1.0,20.0,3.0,1.0,19-35,3
2,4,126,45.086429,30.245,255.36,6,0,0,0,0,...,3,0,2.000000,7,2.0,29.0,4.0,0.0,19-35,other
3,5,9,91.055556,42.500,390.00,0,0,0,0,0,...,0,0,1.285714,2,2.0,22.0,5.0,0.0,19-35,other
4,6,35,226.118857,234.000,496.95,0,0,0,0,0,...,0,0,1.521739,3,1.0,21.0,6.0,1.0,19-35,other
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24855,27764,41,179.386098,170.000,400.92,2,0,0,0,0,...,7,15,1.464286,3,1.0,38.0,295.0,1.0,36-50,other
24856,27765,75,130.001333,110.640,455.00,0,1,1,6,4,...,0,1,1.785714,7,2.0,30.0,79.0,0.0,19-35,other
24857,27766,52,138.974231,120.000,342.00,4,0,0,0,1,...,3,1,1.677419,6,2.0,21.0,1953.0,0.0,19-35,other
24858,27767,1,143.880000,143.880,143.88,0,0,0,0,0,...,0,0,1.000000,1,2.0,17.0,0.0,0.0,0-18,0


In [24]:
# Посмотрим информацию по датасету
user_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24860 entries, 0 to 24859
Data columns (total 37 columns):
 #   Column               Non-Null Count  Dtype   
---  ------               --------------  -----   
 0   user_id              24860 non-null  int64   
 1   total_ads            24860 non-null  int64   
 2   avg_cpm              24860 non-null  float64 
 3   median_cpm           24860 non-null  float64 
 4   max_cpm              24860 non-null  float64 
 5   hour_0               24860 non-null  int64   
 6   hour_1               24860 non-null  int64   
 7   hour_2               24860 non-null  int64   
 8   hour_3               24860 non-null  int64   
 9   hour_4               24860 non-null  int64   
 10  hour_5               24860 non-null  int64   
 11  hour_6               24860 non-null  int64   
 12  hour_7               24860 non-null  int64   
 13  hour_8               24860 non-null  int64   
 14  hour_9               24860 non-null  int64   
 15  hour_10            

## <a href="#26">Создание целевых переменных</a> 

Вот несколько условий по задаче:
- Если ставка объявления `cpm` больше всех ставок всех остальных участников: со 100% вероятность выигрывает объявление со ставкой `cpm`
- Если ставка объявления `cpm` равна максимальной ставке среди всех остальных участников: объявление со ставкой `cpm` выигрывает с вероятностью в 50%. В реальном аукционе объявление может выигрывать по совершенно другим правилам и вероятностям, это упрощение для моделирования конкретно в данном датасете. 

Нам необходимо выявить веротяность выигрыша для каждой ставки

In [25]:
# Сортируем данные для упрощения обработки сессий и вычислений
history = history.sort_values(by=['user_id', 'hour'])

# Определяем max_cpm среди других участников на той же площадке и в тот же час
history['max_cpm_other'] = history.groupby(['hour', 'publisher'])['cpm'].transform(
    lambda x: x.nlargest(2).iloc[-1] if len(x) > 1 else x.iloc[0]
)

In [26]:
history

,hour,cpm,publisher,user_id,hour_of_day,time_diff,new_session,session_id,max_cpm_other
296,8,165.0,1,1,8,0.0,0,0,525.00
972,9,53.2,1,1,9,1.0,0,0,534.36
3297,9,36.0,1,1,9,0.0,0,0,534.36
6377,9,51.0,1,1,9,0.0,0,0,534.36
1049164,19,31.5,1,1,19,10.0,1,1,538.15
...,...,...,...,...,...,...,...,...,...
733215,1188,285.0,2,27768,12,46.0,1,13,531.76
740224,1188,205.0,12,27768,12,0.0,0,13,51.60
988910,1216,210.0,2,27768,16,28.0,1,14,552.35
875868,1410,179.8,2,27768,18,194.0,1,15,517.50


In [27]:
# Вычисляем вероятность выигрыша в аукционе
history['win_probability'] = np.where(
    history['cpm'] > history['max_cpm_other'], 1,  # Победа с вероятностью 1, если ставка больше всех остальных
    np.where(history['cpm'] == history['max_cpm_other'], 0.5, 0)  # Ничья — вероятность 0.5, иначе 0
)

In [28]:
# Формируем уникальные просмотры с учетом правила "не показывать повторно" и сессий (разрыв > 6 часов)
def calculate_unique_views_with_probability(group):
    seen_ads = set()
    unique_views = []
    last_seen_time = {}
    for _, row in group.iterrows():
        user_id = row['user_id']
        if user_id in seen_ads:
            # Учет новой сессии
            if row['hour'] - last_seen_time[user_id] > 6:
                seen_ads.remove(user_id)
            else:
                unique_views.append(0)
                continue
        # Вероятностный учет просмотра
        unique_views.append(row['win_probability'])
        if row['win_probability'] > 0:
            seen_ads.add(user_id)
            last_seen_time[user_id] = row['hour']
    return unique_views

In [29]:
# Применяем к каждому пользователю
history['unique_view'] = (
    history.groupby('user_id', group_keys=False).apply(
        lambda group: pd.Series(calculate_unique_views_with_probability(group))
    ).values  # Конвертируем результат в массив для корректного присоединения
)

/var/folders/01/wgcp1v9n5zjdx14pvmwxt4mc0000gn/T/ipykernel_23089/4162637202.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  history.groupby('user_id', group_keys=False).apply(


In [30]:
history['unique_view'] = history['unique_view'].astype(float)  # Приводим уникальные просмотры к float

In [31]:
history

,hour,cpm,publisher,user_id,hour_of_day,time_diff,new_session,session_id,max_cpm_other,win_probability,unique_view
296,8,165.0,1,1,8,0.0,0,0,525.00,0.0,0.0
972,9,53.2,1,1,9,1.0,0,0,534.36,0.0,0.0
3297,9,36.0,1,1,9,0.0,0,0,534.36,0.0,0.0
6377,9,51.0,1,1,9,0.0,0,0,534.36,0.0,0.0
1049164,19,31.5,1,1,19,10.0,1,1,538.15,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
733215,1188,285.0,2,27768,12,46.0,1,13,531.76,0.0,0.0
740224,1188,205.0,12,27768,12,0.0,0,13,51.60,1.0,1.0
988910,1216,210.0,2,27768,16,28.0,1,14,552.35,0.0,0.0
875868,1410,179.8,2,27768,18,194.0,1,15,517.50,0.0,0.0


In [32]:
# Вычисляем целевые переменные
def calculate_view_targets(group):
    cumsum = group['unique_view'].cumsum()
    return {
        'at_least_one': (cumsum >= 1).mean(),
        'at_least_two': (cumsum >= 2).mean(),
        'at_least_three': (cumsum >= 3).mean()
    }

# Группируем данные по пользователю и применяем подсчет целевых переменных
view_targets = (
    history.groupby('user_id').apply(calculate_view_targets).apply(pd.Series).reset_index()
)

# Переименовываем столбцы для удобства
view_targets.columns = ['user_id', 'at_least_one', 'at_least_two', 'at_least_three']

view_targets

/var/folders/01/wgcp1v9n5zjdx14pvmwxt4mc0000gn/T/ipykernel_23089/3308735392.py:12: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  history.groupby('user_id').apply(calculate_view_targets).apply(pd.Series).reset_index()


,user_id,at_least_one,at_least_two,at_least_three
0,1,0.000000,0.000000,0.0
1,3,1.000000,0.000000,0.0
2,4,0.000000,0.000000,0.0
3,5,0.000000,0.000000,0.0
4,6,0.428571,0.171429,0.0
...,...,...,...,...
24855,27764,0.000000,0.000000,0.0
24856,27765,0.000000,0.000000,0.0
24857,27766,0.000000,0.000000,0.0
24858,27767,0.000000,0.000000,0.0


In [33]:
view_targets.describe()

,user_id,at_least_one,at_least_two,at_least_three
count,24860.000000,24860.000000,24860.000000,24860.000000
mean,13904.366734,0.196475,0.070966,0.037512
std,8020.375554,0.340897,0.203516,0.146608
min,1.000000,0.000000,0.000000,0.000000
25%,6952.750000,0.000000,0.000000,0.000000
50%,13936.500000,0.000000,0.000000,0.000000
75%,20852.250000,0.310181,0.000000,0.000000
max,27768.000000,1.000000,0.995327,0.981443


In [34]:
validate_answers.describe()

,at_least_one,at_least_two,at_least_three
count,1008.000000,1008.000000,1008.000000
mean,0.115441,0.065805,0.047170
std,0.146146,0.117812,0.099029
min,0.000000,0.000000,0.000000
25%,0.016650,0.000000,0.000000
50%,0.055250,0.009800,0.001400
75%,0.158700,0.078125,0.048950
max,0.930700,0.909700,0.883400


In [35]:
# Объединение целевых признаков с user_features
user_features = user_features.merge(view_targets, on='user_id', how='left')

In [36]:
pd.set_option('display.max_columns', None)

In [37]:
user_features

,user_id,total_ads,avg_cpm,median_cpm,max_cpm,hour_0,hour_1,hour_2,hour_3,hour_4,hour_5,hour_6,hour_7,hour_8,hour_9,hour_10,hour_11,hour_12,hour_13,hour_14,hour_15,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23,avg_ads_per_session,max_ads_per_session,sex,age,city_id,sex_binary,age_group,city_group,at_least_one,at_least_two,at_least_three
0,1,79,111.247848,90.000,361.52,0,0,0,0,0,0,0,0,7,15,4,6,3,0,2,9,12,5,4,9,2,0,0,1,2.135135,7,1.0,0.0,1.0,1.0,0-18,other,0.000000,0.000000,0.0
1,3,7,321.588571,263.760,531.22,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,2,1,1,1.000000,1,1.0,20.0,3.0,1.0,19-35,3,1.000000,0.000000,0.0
2,4,126,45.086429,30.245,255.36,6,0,0,0,0,0,2,4,11,19,10,8,6,7,4,4,4,12,6,5,11,4,3,0,2.000000,7,2.0,29.0,4.0,0.0,19-35,other,0.000000,0.000000,0.0
3,5,9,91.055556,42.500,390.00,0,0,0,0,0,0,0,0,0,0,0,1,3,3,1,0,0,0,1,0,0,0,0,0,1.285714,2,2.0,22.0,5.0,0.0,19-35,other,0.000000,0.000000,0.0
4,6,35,226.118857,234.000,496.95,0,0,0,0,0,0,2,4,3,2,4,2,1,1,7,3,2,1,1,0,2,0,0,0,1.521739,3,1.0,21.0,6.0,1.0,19-35,other,0.428571,0.171429,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24855,27764,41,179.386098,170.000,400.92,2,0,0,0,0,0,1,2,1,1,1,0,0,0,0,0,1,0,0,2,2,6,7,15,1.464286,3,1.0,38.0,295.0,1.0,36-50,other,0.000000,0.000000,0.0
24856,27765,75,130.001333,110.640,455.00,0,1,1,6,4,11,6,4,1,6,6,0,3,6,5,0,0,2,6,3,3,0,0,1,1.785714,7,2.0,30.0,79.0,0.0,19-35,other,0.000000,0.000000,0.0
24857,27766,52,138.974231,120.000,342.00,4,0,0,0,1,0,3,2,4,2,2,2,0,2,0,2,2,5,10,3,2,2,3,1,1.677419,6,2.0,21.0,1953.0,0.0,19-35,other,0.000000,0.000000,0.0
24858,27767,1,143.880000,143.880,143.88,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1.000000,1,2.0,17.0,0.0,0.0,0-18,0,0.000000,0.000000,0.0


### Описание столбцов

| **Название столбца**     | **Описание**                                                                 |
|---------------------------|-----------------------------------------------------------------------------|
| `user_id`                | Уникальный идентификатор пользователя.                                      |
| `total_ads`              | Общее количество объявлений, просмотренных пользователем.                  |
| `avg_cpm`                | Средняя стоимость показов рекламы (`cpm`) для пользователя.                |
| `median_cpm`             | Медианная стоимость показов рекламы (`cpm`) для пользователя.              |
| `max_cpm`                | Максимальная стоимость показов рекламы (`cpm`) для пользователя.           |
| `hour_0`, ..., `hour_23` | Количество объявлений, просмотренных пользователем в каждый час суток.      |
| `avg_ads_per_session`    | Среднее количество объявлений, просмотренных в одной сессии.               |
| `max_ads_per_session`    | Максимальное количество объявлений, просмотренных в одной сессии.          |
| `sex`                    | Пол пользователя (1 — мужской, 2 — женский).                               |
| `age`                    | Возраст пользователя.                                                      |
| `city_id`                | Уникальный идентификатор города проживания пользователя.                   |
| `sex_binary`             | Бинарное представление пола (1 — мужской, 0 — женский).                    |
| `age_group`              | Группа возраста пользователя (`0-18`, `19-35`, `36-50`, `51+`).            |
| `city_group`             | Группа города (топ-10 или `other`).                                        |
| `at_least_one`           | Доля показов, где пользователь видел объявление хотя бы один раз.          |
| `at_least_two`           | Доля показов, где пользователь видел объявление хотя бы два раза.          |
| `at_least_three`         | Доля показов, где пользователь видел объявление хотя бы три раза.          |

---

### Дополнительно:
- Столбцы `hour_0`, ..., `hour_23` показывают **количество показов** рекламы в каждом часу для каждого пользователя, а не бинарные значения.
- Целевые переменные `at_least_one`, `at_least_two`, `at_least_three` отражают вероятности, рассчитанные по уникальным просмотрам.

In [38]:
user_features.to_csv('./user_features.csv')